In [1]:
!export PYTHONPATH=${PYTHONPATH}:/home/sai/PythonRobotics/PathPlanning/CubicSpline

In [25]:
from ppo_control import PPOControl
import cubic_spline_planner
import matplotlib.pyplot as plt
import time
import numpy as np
import math
%matplotlib tk

In [26]:
husky_ppo = PPOControl()

In [5]:
ax = [0.0,6.0, 12.5, 14.0, 17.5, 20.0, 25.0] 
ay = [0.0,-3.0, -5.0, -4, 3.0, 2, 0.0]
cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(ax, ay, ds= 0.5)
plt.plot(cx, cy, "+")
plt.show()
cv = [1.0]*len(cx)
husky_ppo.set_waypoints_from_list(cx, cy, cyaw, cv)

In [27]:
#husky_ppo.read_waypoint_file("./unity_waypoints_bkp.txt")
#husky_ppo.read_waypoint_file("ww.txt")
#husky_ppo.read_waypoint_file("./unity_flat.txt")
husky_ppo.read_waypoint_file("./planner_way.txt")
cx = [i[0] for i in husky_ppo.waypoints_list]
cy = [i[1] for i in husky_ppo.waypoints_list]
plt.plot(cx, cy, "+")
plt.show()

In [28]:
def simulate_warthog(X, v, w, dt):
    xcurr = X[0] + v*math.cos(X[2])*dt
    ycurr = X[1] + v*math.sin(X[2])*dt
    thcurr = X[2]+w*dt
    return [xcurr, ycurr, thcurr]

In [29]:
def simulate_warthog2(X, v, w, dt, v_prev, w_prev):
    xcurr = X[0] + v_prev*math.cos(X[2])*dt
    ycurr = X[1] + v_prev*math.sin(X[2])*dt
    thcurr = X[2]+w_prev*dt
    alpha = 1
    beta = 1
    vcurr = v*(alpha) + v_prev*(1-alpha)
    wcurr = w*(alpha) + w_prev*(1-alpha)
    vcurr = v*(alpha)
    wcurr = w*(beta)
    return [xcurr, ycurr, thcurr, vcurr, wcurr]

In [30]:
#husky_ppo.set_pose([30.67, -33.33, 3.06])
#husky_ppo.set_pose([120.180, -78.957, 140*math.pi/180.])
#husky_ppo.set_pose([132.180, -78.957, 160*math.pi/180.])
husky_ppo.set_twist([0. , 0.])
#husky_ppo.set_pose([0. , 0.5, 7*math.pi/4])
start_idx = 0
husky_ppo.set_pose([cx[start_idx]+0.05 , cy[start_idx] + 0.05, husky_ppo.waypoints_list[start_idx][2]])
husky_ppo.read_ppo_policy("./model1.zip")
#husky_ppo.read_tf_frozen_graph("/home/sai/hdd1/ml-master/ml-agents/config/ppo/results/wlong_path52/3DBall/frozen_graph_def.pb")

Using cpu device
Wrapping the env in a DummyVecEnv.


In [31]:
x_pose = []
y_pose = []
v_prev = 0
w_prev = 0
for i in range(0, 2000):
    obs = husky_ppo.get_observation()
    twist = husky_ppo.get_ppo_control(np.array(obs).reshape(1,42))[0]
    print(twist)
    v = np.clip(twist[0][0], 0, 1) * 4.0
    w = np.clip(twist[0][1], -1, 1) * 2.5
    #current_pose = simulate_warthog2(husky_ppo.get_pose(), v, w, 0.05, v_prev, w_prev)
    current_pose = simulate_warthog(husky_ppo.get_pose(), v, w, 0.05)
    print(twist[0])
    closest_idx = husky_ppo.closest_idx
    print(closest_idx, "\n")
    #husky_ppo.set_pose(current_pose[0:3])
    husky_ppo.set_pose(current_pose)
    #v_prev = current_pose[3]
    #w_prev = current_pose[4]
    husky_ppo.set_twist([v, w])
    x_pose.append(current_pose[0])
    y_pose.append(current_pose[1])
    #print(husky_ppo.closest_idx, husky_ppo.closest_dist)

[[1.        0.5079675]]
[1.        0.5079675]
0 

[[ 1. -1.]]
[ 1. -1.]
0 

[[ 0.85347635 -1.        ]]
[ 0.85347635 -1.        ]
1 

[[ 1. -1.]]
[ 1. -1.]
1 

[[ 1. -1.]]
[ 1. -1.]
1 

[[ 1. -1.]]
[ 1. -1.]
1 

[[ 0.5316817 -1.       ]]
[ 0.5316817 -1.       ]
2 

[[1.         0.45797575]]
[1.         0.45797575]
2 

[[-0.22421706  1.        ]]
[-0.22421706  1.        ]
2 

[[1.         0.89060545]]
[1.         0.89060545]
2 

[[-1.         -0.03541162]]
[-1.         -0.03541162]
3 

[[1.         0.76617837]]
[1.         0.76617837]
3 

[[-1.       -0.331863]]
[-1.       -0.331863]
3 

[[1.         0.67958343]]
[1.         0.67958343]
3 

[[-1.         -0.33186173]]
[-1.         -0.33186173]
3 

[[1.        0.6664312]]
[1.        0.6664312]
3 

[[-1.         -0.34720817]]
[-1.         -0.34720817]
3 

[[1.        0.6654579]]
[1.        0.6654579]
3 

[[-0.9042606  -0.87077117]]
[-0.9042606  -0.87077117]
4 

[[1.         0.17293528]]
[1.         0.17293528]
4 

[[-0.9801865 -0.8672971]

[[1.         0.44918898]]
[1.         0.44918898]
43 

[[-1.         -0.75684965]]
[-1.         -0.75684965]
43 

[[1.        0.5856476]]
[1.        0.5856476]
43 

[[-1.        -0.4005406]]
[-1.        -0.4005406]
43 

[[1.         0.69792944]]
[1.         0.69792944]
43 

[[-1.         -0.81031644]]
[-1.         -0.81031644]
44 

[[1.        0.4919535]]
[1.        0.4919535]
44 

[[-1.         -0.72475386]]
[-1.         -0.72475386]
44 

[[1.        0.6185925]]
[1.        0.6185925]
44 

[[-1.        -0.3590356]]
[-1.        -0.3590356]
44 

[[1.         0.71114326]]
[1.         0.71114326]
44 

[[-1.         -0.80051094]]
[-1.         -0.80051094]
45 

[[1.        0.5011757]]
[1.        0.5011757]
45 

[[-1.        -0.7456249]]
[-1.        -0.7456249]
45 

[[1.        0.6209855]]
[1.        0.6209855]
45 

[[-1.         -0.36319962]]
[-1.         -0.36319962]
45 

[[1.         0.71574014]]
[1.         0.71574014]
45 

[[-1.         -0.78901607]]
[-1.         -0.78901607]
46 

[[1.  

[[-1.         -0.76086473]]
[-1.         -0.76086473]
88 

[[1.         0.58651644]]
[1.         0.58651644]
88 

[[-1.        -0.4013914]]
[-1.        -0.4013914]
88 

[[1.        0.6990551]]
[1.        0.6990551]
88 

[[-1.         -0.80848694]]
[-1.         -0.80848694]
89 

[[1.         0.49421477]]
[1.         0.49421477]
89 

[[-1.         -0.72883844]]
[-1.         -0.72883844]
89 

[[1.         0.61938536]]
[1.         0.61938536]
89 

[[-1.        -0.3606276]]
[-1.        -0.3606276]
89 

[[1.         0.71226573]]
[1.         0.71226573]
89 

[[-1.        -0.7982724]]
[-1.        -0.7982724]
90 

[[1.         0.50355715]]
[1.         0.50355715]
90 

[[-1.         -0.74857706]]
[-1.         -0.74857706]
90 

[[1.        0.6215659]]
[1.        0.6215659]
90 

[[-1.         -0.36646482]]
[-1.         -0.36646482]
90 

[[1.        0.7166773]]
[1.        0.7166773]
90 

[[-1.        -0.7865398]]
[-1.        -0.7865398]
91 

[[1.         0.51764965]]
[1.         0.51764965]
91 

[[

[[ 0.5338072 -1.       ]]
[ 0.5338072 -1.       ]
112 

[[ 0.54127556 -1.        ]]
[ 0.54127556 -1.        ]
112 

[[ 0.5492461 -1.       ]]
[ 0.5492461 -1.       ]
112 

[[ 0.5579057 -1.       ]]
[ 0.5579057 -1.       ]
112 

[[ 0.5675456 -1.       ]]
[ 0.5675456 -1.       ]
112 

[[ 0.5781625 -1.       ]]
[ 0.5781625 -1.       ]
112 

[[ 0.58974737 -1.        ]]
[ 0.58974737 -1.        ]
112 

[[ 0.6018834 -1.       ]]
[ 0.6018834 -1.       ]
112 

[[ 0.61419666 -1.        ]]
[ 0.61419666 -1.        ]
112 

[[ 0.62601477 -1.        ]]
[ 0.62601477 -1.        ]
112 

[[ 0.6368669 -1.       ]]
[ 0.6368669 -1.       ]
112 

[[ 0.6459839 -1.       ]]
[ 0.6459839 -1.       ]
112 

[[ 0.6528749 -1.       ]]
[ 0.6528749 -1.       ]
112 

[[ 0.6569386 -1.       ]]
[ 0.6569386 -1.       ]
112 

[[ 0.6583532 -1.       ]]
[ 0.6583532 -1.       ]
112 

[[ 0.65730023 -1.        ]]
[ 0.65730023 -1.        ]
112 

[[ 0.6545989 -1.       ]]
[ 0.6545989 -1.       ]
112 

[[ 0.65055704 -1.        ]]


[[-0.1489605  1.       ]]
[-0.1489605  1.       ]
112 

[[-0.16513416  1.        ]]
[-0.16513416  1.        ]
112 

[[-0.18241528  1.        ]]
[-0.18241528  1.        ]
112 

[[-0.2016575  1.       ]]
[-0.2016575  1.       ]
112 

[[-0.22400731  1.        ]]
[-0.22400731  1.        ]
112 

[[-0.2505317  1.       ]]
[-0.2505317  1.       ]
112 

[[-0.28146362  1.        ]]
[-0.28146362  1.        ]
112 

[[-0.31547374  1.        ]]
[-0.31547374  1.        ]
112 

[[-0.34990755  1.        ]]
[-0.34990755  1.        ]
112 

[[-0.3822747  1.       ]]
[-0.3822747  1.       ]
112 

[[-0.41172436  1.        ]]
[-0.41172436  1.        ]
112 

[[-0.43922275  1.        ]]
[-0.43922275  1.        ]
112 

[[-0.46668655  1.        ]]
[-0.46668655  1.        ]
112 

[[-0.49598837  1.        ]]
[-0.49598837  1.        ]
112 

[[-0.52827346  1.        ]]
[-0.52827346  1.        ]
112 

[[-0.5635985  1.       ]]
[-0.5635985  1.       ]
112 

[[-0.60084075  1.        ]]
[-0.60084075  1.        ]
112 



...

[[-1.  1.]]
[-1.  1.]
112 

[[-1.         0.9771169]]
[-1.         0.9771169]
112 

[[-1.          0.86151296]]
[-1.          0.86151296]
112 

[[-1.         0.5916505]]
[-1.         0.5916505]
112 

[[-1.          0.17602906]]
[-1.          0.17602906]
112 

[[-1.         -0.57654333]]
[-1.         -0.57654333]
112 

[[-0.39520252 -1.        ]]
[-0.39520252 -1.        ]
112 

[[-0.24741465 -1.        ]]
[-0.24741465 -1.        ]
112 

[[-0.26961517 -1.        ]]
[-0.26961517 -1.        ]
112 

[[-0.29511923 -1.        ]]
[-0.29511923 -1.        ]
112 

[[-0.3236534 -1.       ]]
[-0.3236534 -1.       ]
112 

[[-0.35452712 -1.        ]]
[-0.35452712 -1.        ]
112 

[[-0.3866399 -1.       ]]
[-0.3866399 -1.       ]
112 

[[-0.41854513 -1.        ]]
[-0.41854513 -1.        ]
112 

[[-0.44852197 -1.        ]]
[-0.44852197 -1.        ]
112 

[[-0.47459728 -1.        ]]
[-0.47459728 -1.        ]
112 

[[-0.4944951 -1.       ]]
[-0.4944951 -1.       ]
112 

[[-0.5055519 -1.       ]]
[-0.50

[[ 0.3568388 -0.7860637]]
[ 0.3568388 -0.7860637]
112 

[[-0.47266692 -0.5926533 ]]
[-0.47266692 -0.5926533 ]
112 

[[-0.03904912  0.0348638 ]]
[-0.03904912  0.0348638 ]
112 

[[-0.36376044  1.        ]]
[-0.36376044  1.        ]
112 

[[-0.40588114  1.        ]]
[-0.40588114  1.        ]
112 

[[-0.42537892  1.        ]]
[-0.42537892  1.        ]
112 

[[-0.4449647  1.       ]]
[-0.4449647  1.       ]
112 

[[-0.46465725  1.        ]]
[-0.46465725  1.        ]
112 

[[-0.48454434  1.        ]]
[-0.48454434  1.        ]
112 

[[-0.5047575  1.       ]]
[-0.5047575  1.       ]
112 

[[-0.5254534  1.       ]]
[-0.5254534  1.       ]
112 

[[-0.54680824  1.        ]]
[-0.54680824  1.        ]
112 

[[-0.5690292  1.       ]]
[-0.5690292  1.       ]
112 

[[-0.59237677  1.        ]]
[-0.59237677  1.        ]
112 

[[-0.6171959  1.       ]]
[-0.6171959  1.       ]
112 

[[-0.64395076  1.        ]]
[-0.64395076  1.        ]
112 

[[-0.6732614  1.       ]]
[-0.6732614  1.       ]
112 

[[-0.705

[[-0.8372906 -1.       ]]
[-0.8372906 -1.       ]
112 

[[-0.7999488 -1.       ]]
[-0.7999488 -1.       ]
112 

[[-0.7476297 -1.       ]]
[-0.7476297 -1.       ]
112 

[[-0.680505 -1.      ]]
[-0.680505 -1.      ]
112 

[[-0.59907764 -1.        ]]
[-0.59907764 -1.        ]
112 

[[-0.5041048 -1.       ]]
[-0.5041048 -1.       ]
112 

[[-0.39734197 -1.        ]]
[-0.39734197 -1.        ]
112 

[[-0.28249788 -1.        ]]
[-0.28249788 -1.        ]
112 

[[-0.16528703 -1.        ]]
[-0.16528703 -1.        ]
112 

[[-0.05240043 -1.        ]]
[-0.05240043 -1.        ]
112 

[[ 0.0500766 -1.       ]]
[ 0.0500766 -1.       ]
112 

[[ 0.06903706 -1.        ]]
[ 0.06903706 -1.        ]
112 

[[ 0.09977819 -0.9403261 ]]
[ 0.09977819 -0.9403261 ]
112 

[[ 0.02142785 -0.782045  ]]
[ 0.02142785 -0.782045  ]
112 

[[-0.00312164 -0.45018464]]
[-0.00312164 -0.45018464]
112 

[[-0.28868148  0.2860897 ]]
[-0.28868148  0.2860897 ]
112 

[[-0.4525786  1.       ]]
[-0.4525786  1.       ]
112 

[[-0.4440937

[[-0.8135442 -1.       ]]
[-0.8135442 -1.       ]
112 

[[-0.842187 -1.      ]]
[-0.842187 -1.      ]
112 

[[-0.86687165 -1.        ]]
[-0.86687165 -1.        ]
112 

[[-0.8862167 -1.       ]]
[-0.8862167 -1.       ]
112 

[[-0.89857507 -1.        ]]
[-0.89857507 -1.        ]
112 

[[-0.9021444 -1.       ]]
[-0.9021444 -1.       ]
112 

[[-0.8950874 -1.       ]]
[-0.8950874 -1.       ]
112 

[[-0.875706 -1.      ]]
[-0.875706 -1.      ]
112 

[[-0.8427011 -1.       ]]
[-0.8427011 -1.       ]
112 

[[-0.79541105 -1.        ]]
[-0.79541105 -1.        ]
112 

[[-0.7337976 -1.       ]]
[-0.7337976 -1.       ]
112 

[[-0.658141 -1.      ]]
[-0.658141 -1.      ]
112 

[[-0.568935 -1.      ]]
[-0.568935 -1.      ]
112 

[[-0.46752125 -1.        ]]
[-0.46752125 -1.        ]
112 

[[-0.35703874 -1.        ]]
[-0.35703874 -1.        ]
112 

[[-0.24262664 -1.        ]]
[-0.24262664 -1.        ]
112 

[[-0.13058981 -1.        ]]
[-0.13058981 -1.        ]
112 

[[-0.02700481 -1.        ]]
[-0.0270

[[-1.         -0.04384517]]
[-1.         -0.04384517]
112 

[[-1.         -0.40216145]]
[-1.         -0.40216145]
112 

[[-1. -1.]]
[-1. -1.]
112 

[[-0.79165626 -1.        ]]
[-0.79165626 -1.        ]
112 

[[-0.8234995 -1.       ]]
[-0.8234995 -1.       ]
112 

[[-0.8526214 -1.       ]]
[-0.8526214 -1.       ]
112 

[[-0.8780498 -1.       ]]
[-0.8780498 -1.       ]
112 

[[-0.8984706 -1.       ]]
[-0.8984706 -1.       ]
112 

[[-0.91231126 -1.        ]]
[-0.91231126 -1.        ]
112 

[[-0.91784805 -1.        ]]
[-0.91784805 -1.        ]
112 

[[-0.9133115 -1.       ]]
[-0.9133115 -1.       ]
112 

[[-0.89703035 -1.        ]]
[-0.89703035 -1.        ]
112 

[[-0.8676571 -1.       ]]
[-0.8676571 -1.       ]
112 

[[-0.82440406 -1.        ]]
[-0.82440406 -1.        ]
112 

[[-0.767082 -1.      ]]
[-0.767082 -1.      ]
112 

[[-0.69584626 -1.        ]]
[-0.69584626 -1.        ]
112 

[[-0.6110339 -1.       ]]
[-0.6110339 -1.       ]
112 

[[-0.5136632 -1.       ]]
[-0.5136632 -1.       

In [32]:
plt.cla()
plt.plot(x_pose, y_pose, '+g')
plt.plot(cx, cy, "+b")
#plt.xlim([0, 25])
#plt.ylim([-20, 7])

In [ ]:
obs = husky_ppo.get_observation()

In [ ]:
print(obs)

In [ ]:
print(husky_ppo.get_pose())

In [ ]:
husky_ppo.closest_idx

In [ ]:
2*math.pi - 6.1321

In [ ]:
i = 0
while i < len(obs)-3:
    print([obs[i], obs[i+1], obs[i+2], obs[i+3]])
    i = i + 4

In [ ]:
obs2 = [ 3.1718987e-01,  2.6397274e+00, -7.1165800e-02,  2.9253261e+00,
         8.2946199e-01,  2.4810174e+00, -2.3757100e-01,  2.9868939e+00,
         1.4194126e+00,  2.3704438e+00, -3.9009976e-01,  2.9857869e+00,
         2.0244434e+00,  2.2775273e+00, -4.8208833e-01,  3.2309980e+00,
         2.5908678e+00,  2.2095571e+00, -5.5408025e-01,  3.0272779e+00,
         3.1385303e+00,  2.1544523e+00, -5.6134295e-01,  2.8097680e+00,
         3.6512144e+00,  2.1172132e+00, -5.9754920e-01,  2.6893539e+00,
         4.2099533e+00,  2.0821514e+00, -6.1818290e-01,  2.6712611e+00,
         4.7472596e+00,  2.0539546e+00, -6.3812470e-01,  2.9293561e+00,
         5.2431216e+00,  2.0312028e+00, -6.4151406e-01,  2.9405370e+00,
         3.7776546e-03, -1.4553810e-03]

In [ ]:
i = 0
while i < len(obs2)-3:
    print([obs2[i], obs2[i+1], obs2[i+2], obs2[i+3]])
    i = i + 4

In [ ]:
print(husky_ppo.num_waypoints)

In [20]:
print(husky_ppo.waypoints_list)

[array([0.        , 0.        , 5.84690706, 4.        ]), array([ 0.43545074, -0.20302535,  5.84595374,  4.        ]), array([ 0.87129576, -0.40674059,  5.84405479,  4.        ]), array([ 1.30792936, -0.61183558,  5.8412255 ,  4.        ]), array([ 1.74574581, -0.81900022,  5.83748844,  4.        ]), array([ 2.1851394 , -1.02892438,  5.83287314,  4.        ]), array([ 2.62650443, -1.24229795,  5.82741558,  4.        ]), array([ 3.07023516, -1.4598108 ,  5.82115759,  4.        ]), array([ 3.5167259 , -1.68215281,  5.81414619,  4.        ]), array([ 3.96637093, -1.91001388,  5.80643284,  4.        ]), array([ 4.41956452, -2.14408387,  5.79807264,  4.        ]), array([ 4.87670098, -2.38505267,  5.78912349,  4.        ]), array([ 5.33817457, -2.63361015,  5.77964528,  4.        ]), array([ 5.8043796 , -2.89044621,  5.77000864,  4.        ]), array([ 6.27566617, -3.15603351,  5.76509605,  4.        ]), array([ 6.75167852, -3.42737363,  5.76856828,  4.        ]), array([ 7.23149358, -3.6986